In [1]:
#Imporing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# Reading Data Files

#tdcsfog file
tdcsfog_test = pd.read_csv('tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/003f117e14.csv')
tdcsfog_test.head(5)

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
0,0,-9.533939,0.566322,-1.413525,0,0,0
1,1,-9.536140,0.564137,-1.440621,0,0,0
2,2,-9.529345,0.561765,-1.429332,0,0,0
3,3,-9.531239,0.564227,-1.415490,0,0,0
4,4,-9.540825,0.561854,-1.429471,0,0,0


In [3]:
# tdcsfog metadata file
tdcsfog_meta = pd.read_csv("tlvmc-parkinsons-freezing-gait-prediction/tdcsfog_metadata.csv")
tdcsfog_meta.head(5)

,Id,Subject,Visit,Test,Medication
0,003f117e14,13abfd,3,2,on
1,009ee11563,d81e3a,4,2,on
2,011322847a,203e85,2,2,on
3,01d0fe7266,203e85,2,1,off
4,024418ba39,cecfb8,19,3,on


In [4]:
# Read defog data
defog_test = pd.read_csv('tlvmc-parkinsons-freezing-gait-prediction/train/defog/02ea782681.csv')
defog_test.head(5)

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Valid,Task
0,0,-1.0,0.044129,-0.25,0,0,0,False,False
1,1,-1.0,0.034431,-0.25,0,0,0,False,False
2,2,-1.0,0.031250,-0.25,0,0,0,False,False
3,3,-1.0,0.031250,-0.25,0,0,0,False,False
4,4,-1.0,0.031250,-0.25,0,0,0,False,False


In [5]:
# Defog metadata
defog_meta = pd.read_csv("tlvmc-parkinsons-freezing-gait-prediction/defog_metadata.csv")
defog_meta.head(5)

,Id,Subject,Visit,Medication
0,02ab235146,ab54e1,2,on
1,02ea782681,bf608b,2,on
2,06414383cf,c0b71e,2,off
3,092b4c1819,b6a627,1,off
4,0a900ed8a2,b7bd52,2,on


In [6]:
#Subject
subject_df = pd.read_csv("tlvmc-parkinsons-freezing-gait-prediction/subjects.csv")
subject_df.head(5)

,Subject,Visit,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ
0,04fcdb,1.0,63,M,3.0,30.0,NaN,0
1,05595e,1.0,56,M,8.0,28.0,NaN,0
2,0967b2,1.0,59,M,10.0,38.0,48.0,19
3,0967b2,2.0,59,M,10.0,37.0,44.0,13
4,097078,NaN,70,F,10.0,27.0,50.0,20


In [7]:
defog_meta['Type'] = "defog"
tdcsfog_meta['Type'] = "tdcsfog"

In [8]:
# Combine metadata

metadata=pd.concat([defog_meta,tdcsfog_meta])
metadata.fillna(0,inplace=True)
metadata

,Id,Subject,Visit,Medication,Type,Test
0,02ab235146,ab54e1,2,on,defog,0.0
1,02ea782681,bf608b,2,on,defog,0.0
2,06414383cf,c0b71e,2,off,defog,0.0
3,092b4c1819,b6a627,1,off,defog,0.0
4,0a900ed8a2,b7bd52,2,on,defog,0.0
...,...,...,...,...,...,...
828,feba449e1a,47860d,19,on,tdcsfog,1.0
829,ff4f844fd3,43fcae,2,on,tdcsfog,3.0
830,ff53514514,a2a051,2,on,tdcsfog,3.0
831,ff92d9244d,a9e866,20,on,tdcsfog,2.0


In [9]:
metadata['Medication'] = metadata['Medication'].map( {'on': 0, 'off': 1} ).astype(int)

In [10]:
metadata_merged = metadata.merge(subject_df,how='left',on='Subject').copy()
metadata_merged.fillna(0, inplace=True)

In [11]:
metadata_merged['Sex'] = metadata_merged['Sex'].map( {'M': 0, 'F': 1} ).astype(int)

In [12]:
metadata_merged

,Id,Subject,Visit_x,Medication,Type,Test,Visit_y,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ
0,02ab235146,ab54e1,2,0,defog,0.0,1.0,79,1,8.0,52.0,55.0,28
1,02ab235146,ab54e1,2,0,defog,0.0,2.0,79,1,8.0,38.0,64.0,28
2,02ea782681,bf608b,2,0,defog,0.0,1.0,67,0,7.0,15.0,48.0,23
3,02ea782681,bf608b,2,0,defog,0.0,2.0,67,0,7.0,14.0,57.0,19
4,06414383cf,c0b71e,2,1,defog,0.0,1.0,61,1,7.0,45.0,52.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078,feba449e1a,47860d,19,0,tdcsfog,1.0,0.0,79,1,6.0,44.0,0.0,22
1079,ff4f844fd3,43fcae,2,0,tdcsfog,3.0,0.0,76,0,5.0,41.0,0.0,20
1080,ff53514514,a2a051,2,0,tdcsfog,3.0,0.0,51,1,3.0,29.0,37.0,21
1081,ff92d9244d,a9e866,20,0,tdcsfog,2.0,0.0,74,0,4.0,46.0,44.0,25


In [13]:
metadata_merged.drop(['YearsSinceDx', 'UPDRSIII_On', 'UPDRSIII_Off', 'NFOGQ', 'Visit_x', 'Visit_y', 'Test'], axis=1, inplace=True)
metadata_merged

,Id,Subject,Medication,Type,Age,Sex
0,02ab235146,ab54e1,0,defog,79,1
1,02ab235146,ab54e1,0,defog,79,1
2,02ea782681,bf608b,0,defog,67,0
3,02ea782681,bf608b,0,defog,67,0
4,06414383cf,c0b71e,1,defog,61,1
...,...,...,...,...,...,...
1078,feba449e1a,47860d,0,tdcsfog,79,1
1079,ff4f844fd3,43fcae,0,tdcsfog,76,0
1080,ff53514514,a2a051,0,tdcsfog,51,1
1081,ff92d9244d,a9e866,0,tdcsfog,74,0


In [14]:
bins = [0,25,50,75,100]
labels = [0,1,2,3]
metadata_merged['Age_group'] = pd.cut(metadata_merged['Age'],bins,labels=labels)

In [15]:
metadata_merged = metadata_merged.drop_duplicates()
metadata_merged = metadata_merged.reset_index(drop=True)
metadata_merged

,Id,Subject,Medication,Type,Age,Sex,Age_group
0,02ab235146,ab54e1,0,defog,79,1,3
1,02ea782681,bf608b,0,defog,67,0,2
2,06414383cf,c0b71e,1,defog,61,1,2
3,092b4c1819,b6a627,1,defog,68,1,2
4,0a900ed8a2,b7bd52,0,defog,62,0,2
...,...,...,...,...,...,...,...
969,feba449e1a,47860d,0,tdcsfog,79,1,3
970,ff4f844fd3,43fcae,0,tdcsfog,76,0,3
971,ff53514514,a2a051,0,tdcsfog,51,1,2
972,ff92d9244d,a9e866,0,tdcsfog,74,0,2


In [16]:
import glob
path="tlvmc-parkinsons-freezing-gait-prediction/"

defog_train = glob.glob(path+'train/defog/**')
tdcsfog_train = glob.glob(path+'train/tdcsfog/**')

In [17]:
def get_data_from_csv(f):
    df = pd.read_csv(f)
    df['Id'] = f.split('\\')[-1].split('.')[0]
    return df

In [18]:
defog_train_df = pd.concat([get_data_from_csv(f) for f in defog_train])
defog_train_df['Type'] = 'defog'
tdcsfog_train_df = pd.concat([get_data_from_csv(f) for f in tdcsfog_train])
tdcsfog_train_df['Type'] = 'tdcsfog'

In [19]:
df=pd.concat([defog_train_df,tdcsfog_train_df])
# df = df.fillna(0)
df

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Valid,Task,Id,Type
0,0,-1.000000,0.044129,-0.250000,0,0,0,False,False,02ea782681,defog
1,1,-1.000000,0.034431,-0.250000,0,0,0,False,False,02ea782681,defog
2,2,-1.000000,0.031250,-0.250000,0,0,0,False,False,02ea782681,defog
3,3,-1.000000,0.031250,-0.250000,0,0,0,False,False,02ea782681,defog
4,4,-1.000000,0.031250,-0.250000,0,0,0,False,False,02ea782681,defog
...,...,...,...,...,...,...,...,...,...,...,...
4220,4220,-9.403467,0.089003,-3.220304,0,0,0,NaN,NaN,ffda8fadfd,tdcsfog
4221,4221,-9.404246,0.090531,-3.216584,0,0,0,NaN,NaN,ffda8fadfd,tdcsfog
4222,4222,-9.405770,0.084380,-3.224039,0,0,0,NaN,NaN,ffda8fadfd,tdcsfog
4223,4223,-9.403579,0.084236,-3.236686,0,0,0,NaN,NaN,ffda8fadfd,tdcsfog


In [20]:
combined_df = df.merge(metadata_merged,how='left',on='Id').copy()
combined_df

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Valid,Task,Id,Type_x,Subject,Medication,Type_y,Age,Sex,Age_group
0,0,-1.000000,0.044129,-0.250000,0,0,0,False,False,02ea782681,defog,bf608b,0,defog,67,0,2
1,1,-1.000000,0.034431,-0.250000,0,0,0,False,False,02ea782681,defog,bf608b,0,defog,67,0,2
2,2,-1.000000,0.031250,-0.250000,0,0,0,False,False,02ea782681,defog,bf608b,0,defog,67,0,2
3,3,-1.000000,0.031250,-0.250000,0,0,0,False,False,02ea782681,defog,bf608b,0,defog,67,0,2
4,4,-1.000000,0.031250,-0.250000,0,0,0,False,False,02ea782681,defog,bf608b,0,defog,67,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21093117,4220,-9.403467,0.089003,-3.220304,0,0,0,NaN,NaN,ffda8fadfd,tdcsfog,bae0ce,1,tdcsfog,65,0,2
21093118,4221,-9.404246,0.090531,-3.216584,0,0,0,NaN,NaN,ffda8fadfd,tdcsfog,bae0ce,1,tdcsfog,65,0,2
21093119,4222,-9.405770,0.084380,-3.224039,0,0,0,NaN,NaN,ffda8fadfd,tdcsfog,bae0ce,1,tdcsfog,65,0,2
21093120,4223,-9.403579,0.084236,-3.236686,0,0,0,NaN,NaN,ffda8fadfd,tdcsfog,bae0ce,1,tdcsfog,65,0,2


In [21]:
combined_df.drop(['Valid', 'Task', 'Type_y', 'Subject'], axis=1, inplace=True)
combined_df

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Id,Type_x,Medication,Age,Sex,Age_group
0,0,-1.000000,0.044129,-0.250000,0,0,0,02ea782681,defog,0,67,0,2
1,1,-1.000000,0.034431,-0.250000,0,0,0,02ea782681,defog,0,67,0,2
2,2,-1.000000,0.031250,-0.250000,0,0,0,02ea782681,defog,0,67,0,2
3,3,-1.000000,0.031250,-0.250000,0,0,0,02ea782681,defog,0,67,0,2
4,4,-1.000000,0.031250,-0.250000,0,0,0,02ea782681,defog,0,67,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21093117,4220,-9.403467,0.089003,-3.220304,0,0,0,ffda8fadfd,tdcsfog,1,65,0,2
21093118,4221,-9.404246,0.090531,-3.216584,0,0,0,ffda8fadfd,tdcsfog,1,65,0,2
21093119,4222,-9.405770,0.084380,-3.224039,0,0,0,ffda8fadfd,tdcsfog,1,65,0,2
21093120,4223,-9.403579,0.084236,-3.236686,0,0,0,ffda8fadfd,tdcsfog,1,65,0,2


In [22]:
combined_df['Type_x'] = combined_df['Type_x'].map( {'defog': 0, 'tdcsfog': 1} ).astype(int)
combined_df

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Id,Type_x,Medication,Age,Sex,Age_group
0,0,-1.000000,0.044129,-0.250000,0,0,0,02ea782681,0,0,67,0,2
1,1,-1.000000,0.034431,-0.250000,0,0,0,02ea782681,0,0,67,0,2
2,2,-1.000000,0.031250,-0.250000,0,0,0,02ea782681,0,0,67,0,2
3,3,-1.000000,0.031250,-0.250000,0,0,0,02ea782681,0,0,67,0,2
4,4,-1.000000,0.031250,-0.250000,0,0,0,02ea782681,0,0,67,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21093117,4220,-9.403467,0.089003,-3.220304,0,0,0,ffda8fadfd,1,1,65,0,2
21093118,4221,-9.404246,0.090531,-3.216584,0,0,0,ffda8fadfd,1,1,65,0,2
21093119,4222,-9.405770,0.084380,-3.224039,0,0,0,ffda8fadfd,1,1,65,0,2
21093120,4223,-9.403579,0.084236,-3.236686,0,0,0,ffda8fadfd,1,1,65,0,2


In [23]:
def reduce_memory_usage(df):
    
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()

                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        pass
            else:
                df[col] = df[col].astype('category')
    mem_usg = df.memory_usage().sum() / 1024 ** 2 
    print("Memory usage became: ",mem_usg," MB")
    
    return df

In [24]:
combined_df = reduce_memory_usage(combined_df)

Memory usage of dataframe is 1870.79 MB
Memory usage became:  563.2859001159668  MB


In [25]:
combined_df

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Id,Type_x,Medication,Age,Sex,Age_group
0,0,-1.00000,0.044128,-0.250000,0,0,0,02ea782681,0,0,67,0,2
1,1,-1.00000,0.034424,-0.250000,0,0,0,02ea782681,0,0,67,0,2
2,2,-1.00000,0.031250,-0.250000,0,0,0,02ea782681,0,0,67,0,2
3,3,-1.00000,0.031250,-0.250000,0,0,0,02ea782681,0,0,67,0,2
4,4,-1.00000,0.031250,-0.250000,0,0,0,02ea782681,0,0,67,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21093117,4220,-9.40625,0.088989,-3.220703,0,0,0,ffda8fadfd,1,1,65,0,2
21093118,4221,-9.40625,0.090515,-3.216797,0,0,0,ffda8fadfd,1,1,65,0,2
21093119,4222,-9.40625,0.084351,-3.224609,0,0,0,ffda8fadfd,1,1,65,0,2
21093120,4223,-9.40625,0.084229,-3.236328,0,0,0,ffda8fadfd,1,1,65,0,2


In [26]:
# X=['Time', 'AccV', 'AccML', 'AccAP', 'Type_x', 'Medication', 'Sex', 'Age_group']
# y=['StartHesitation', 'Turn' , 'Walking']
X=['Time', 'AccV', 'AccML', 'AccAP', 'Medication', 'Sex', 'Age']
y=['StartHesitation', 'Turn' , 'Walking']

In [27]:
# Spllting Datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(combined_df[X], combined_df[y],
                                                                      test_size=.30, random_state=15)

In [28]:
#Training Model (RF)
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(max_depth= 5, min_samples_leaf=1, min_samples_split=2, n_estimators=50,
                           max_features='sqrt', random_state=15, n_jobs=-1)
rf_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=50,
                      n_jobs=-1, random_state=15)

In [29]:
#Testing
from sklearn.metrics import average_precision_score
y_prediction = rf_model.predict(X_test)
print(average_precision_score(y_test, y_prediction.clip(0.0,1.0)))

0.48465519169817833


In [35]:
test_data_files = glob.glob(path + 'test/**/**')
submissions = []
for file in test_data_files:
    test_df = pd.read_csv(file)
    org_time = test_df[['Time']]
    test_df = test_df.assign(Id=file.split('\\')[-1].split('.')[0])
    if 'defog' in file:
        test_df = test_df.assign(data_type=1)
    else:
        test_df = test_df.assign(data_type=0)
    df_test = test_df.merge(metadata_merged, how='left', on='Id').copy()
    results = pd.DataFrame(np.round(rf_model.predict(df_test[X]),3), columns=y)
    df_temp = df_test[['Id']]
    org_time = org_time.reset_index(drop=True)
    results = results.reset_index(drop=True)
    sub = pd.concat([org_time, results], axis=1)
    sub = pd.concat([df_temp, sub], axis=1)
    sub['Id'] = sub['Id'].astype(str) + '_' + sub['Time'].astype(str)
    submissions.append(sub[['Id','StartHesitation', 'Turn' , 'Walking']])
    
submission = pd.concat(submissions)
submission[['Id','StartHesitation', 'Turn' , 'Walking']].to_csv('submission.csv', index=False)